In [3]:
# %%
import cv2
import numpy as np
import os
import pandas as pd

from itertools import takewhile
from scipy.spatial import distance_matrix
from seaborn import color_palette

from flytracker.io import DataLoader
from flytracker.analysis.annotating import *

In [20]:
movie_path = ("/home/gert-jan/Documents/flyTracker/data/experiments/bruno/videos/seq_1.mp4")
data_path = "/home/gert-jan/Documents/flyTracker/data/experiments/bruno/results/df.hdf"
output_loc = "/home/gert-jan/Documents/flyTracker/data/experiments/bruno/videos/annotated_video.mp4"
df = pd.read_hdf(data_path, key="df")

In [21]:
cv2.VideoCapture(movie_path).read()[1].shape

(1080, 1280, 3)

In [22]:
cv2.VideoCapture(movie_path).read()[1].dtype

dtype('uint8')

In [28]:
track_length=30
touching_distance=10
image_size=(1280, 1080)#(1080, 1280)
max_frames=100

In [29]:
# Parsing dataframe to numpy array - much faster
data, n_flies_per_arena = parse_data(df)
track_length = int(np.around(track_length * 30))
color_fn = lambda ID: color_picker(ID, n_flies_per_arena)

# Making dataset
initial_frame = data[0, 0, 0]
n_frames = data.shape[0] if max_frames is None else max_frames

assert movie_path.split(".")[-1] == "mp4", "Movie should be mp4."
loader = DataLoader(movie_path, parallel=False)
# plus 1 for intiial frame since we plot (n-1, n)
loader.dataset.set_frame(initial_frame + 1)

# Setting up loader and writer
writer = setup_writer(output_loc, image_size, fps=30)

In [30]:
reader = cv2.VideoCapture(movie_path)

In [31]:
for idx, image in takewhile(lambda x: x[0] < n_frames, enumerate(loader, start=1)):
    #image = image.numpy().squeeze()
    #print(image.shape, image.dtype)
    image = reader.read()[1]
    #lower_frame, upper_frame = np.maximum(idx - track_length, 0), idx

    #image = add_frame_info(image, f"frame: {upper_frame}")
    # First write tracks so that numbers don't get occluded.
    #image = write_tracks(image, data[lower_frame:upper_frame], color_fn)
    #image = write_ID(image, data[upper_frame], touching_distance=touching_distance)
    #print(image.shape, image.dtype)
    writer.write(image)
writer.release()

In [ ]:


def annotate(
    df,
    movie_path,
    output_loc,
    max_frames=None,
    track_length=30,
    touching_distance=10,
    image_size=(1080, 1280),
):

    # Parsing dataframe to numpy array - much faster
    data, n_flies_per_arena = parse_data(df)
    track_length = int(np.around(track_length * 30))
    color_fn = lambda ID: color_picker(ID, n_flies_per_arena)

    # Making dataset
    initial_frame = data[0, 0, 0]
    n_frames = data.shape[0] if max_frames is None else max_frames

    assert movie_path.split(".")[-1] == "mp4", "Movie should be mp4."
    loader = DataLoader(movie_path, parallel=False)
    # plus 1 for intiial frame since we plot (n-1, n)
    loader.dataset.set_frame(initial_frame + 1)

    # Setting up loader and writer
    writer = setup_writer(output_loc, image_size, fps=30)

    for idx, image in takewhile(lambda x: x[0] < n_frames, enumerate(loader, start=1)):
        image = image.numpy().squeeze()
        lower_frame, upper_frame = np.maximum(idx - track_length, 0), idx

        image = add_frame_info(image, f"frame: {upper_frame}")
        # First write tracks so that numbers don't get occluded.
        image = write_tracks(image, data[lower_frame:upper_frame], color_fn)
        image = write_ID(image, data[upper_frame], touching_distance=touching_distance)
        print(image.shape, image.dtype)
        writer.write(image)

        if idx % 1000 == 0:
            print(f"Done with frame {idx}")
    writer.release()

    # Compressing to h264 with ffmpeg
    compressed_loc = output_loc.split(".")[0] + "_compressed.mp4"
    os.system(f"ffmpeg -i {output_loc} -an -vcodec libx264 -crf 23 {compressed_loc}")
